In [1]:
############################################################################################################
# Overview: This script encodes the extracted patient-level data mart.
############################################################################################################

In [1]:
############################################################################################################
# Import packages
############################################################################################################
import datetime
import numpy as np
import os
import pandas as pd
import pyarrow.parquet as pq
import warnings
from ast import literal_eval
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [6]:
########################################################################################################################
# USER_SPECIFIC SETTING
# DATA_IN_DIR_PATH: Path of the input directory of the encounter-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DICT_IN_FILE_PATH: Path of the input data dictionary file
# (created in C02_Encode_Patient_Variables.ipynb)
# DICT_OUT_FILE_PATH: Path of the output data dictionary file
########################################################################################################################
DATA_IN_DIR_PATH: str = '../00_Data/00_Raw_Data/'
DICT_IN_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v1.xlsx'
DICT_OUT_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v2.xlsx'

In [ ]:
########################################################################################################################
# Extract the variables in the data
########################################################################################################################
enc_path: str = os.path.join(DATA_IN_DIR_PATH, 'Encounter_full.parquet')
pf = pq.ParquetFile(enc_path)
enc_vars: list[str] = pf.schema.names
enc_vars = [c for c in enc_vars if c not in ['EncounterKey', 'PatientDurableKey']]

In [3]:
########################################################################################################################
# Create a data dictionary in pandas.DataFrame format
########################################################################################################################
df_enc_dict: pd.DataFrame = pd.DataFrame({'Variable_Name': enc_vars})
df_enc_dict['Encoded_Values'] = np.nan
df_enc_dict['Variable_Type'] = np.nan
df_enc_dict['Remark'] = np.nan

In [ ]:
########################################################################################################################
# 1. Encode date variables
########################################################################################################################
date_cols: list[str] = [enc_vars[i] for i in [1, 33, 36, 39, 42, 45, 48, 51, 77, 79, 81, 83]]
assert all(col.endswith('DateKey') for col in date_cols)
for date_col in date_cols:
    if date_col == 'FatigueDateKey':
        data_type = 'No Data'
    elif date_col == 'EncDateKey':
        data_type = 'Date as integer'
    else:
        data_type = 'Date as float'
    df_enc_dict.loc[df_enc_dict['Variable_Name'] == date_col, 'Variable_Type'] = data_type

In [ ]:
########################################################################################################################
# 2. Encode year variables
########################################################################################################################
year_cols: list[str] = [enc_vars[i] for i in range(87, 94)]
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(year_cols), 'Variable_Type'] = 'Year as float'

In [ ]:
########################################################################################################################
# 3. Variables with no data
########################################################################################################################
empty_cols: list[str] = ['FatigueScore']
for c in empty_cols:
    df: pd.DataFrame = pd.read_parquet(enc_path, columns=[c])
    assert df[c].notna().sum() == 0
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(empty_cols), 'Variable_Type'] = 'No Data'

In [ ]:
########################################################################################################################
# 4. Encode binary variables
########################################################################################################################
bin_cols: list[str] = [enc_vars[i] for i in list(range(7, 11)) + list(range(13, 18)) + list(range(22, 31)) + list(range(52, 74)) + [85, 86, 94, 97]]
for b in bin_cols:
    df: pd.DataFrame = pd.read_parquet(enc_path, columns=[b])
    assert set(df[b].unique().tolist()) == {'Y', 'N'}
df_enc_dict.loc[df_pat_dict['Variable_Name'].isin(bin_cols), 'Variable_Type'] = 'Binary'
df_enc_dict.loc[df_pat_dict['Variable_Name'].isin(bin_cols), 'Encoded_Values'] = str({'N': 0, 'Y': 1})

In [ ]:
########################################################################################################################
# 5. Encoding encounter date
########################################################################################################################
df_enc_dict.loc[df_enc_dict['Variable_Name'] == 'EncDate', 'Variable_Type'] = 'datetime.date'

In [ ]:
########################################################################################################################
# 6. Continuous variables encoding
########################################################################################################################
cont_cols: list[str] = [enc_vars[i] for i in [2, 3, 4, 5, 6, 11, 12, 18, 19, 31, 34, 37, 40, 43, 46, 49, 74, 76, 78, 82, 84]]
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(cont_cols), 'Variable_Type'] = 'Continuous'

In [ ]:
########################################################################################################################
# 7. Encoding ordinal variables with the same encoding scheme
########################################################################################################################
ord_cols: list[str] = [enc_vars[i] for i in range(32, 51, 3)]
encode: dict = {'*Unspecified': None, 'Low Risk': 0, 'Medium Risk': 1, 'Medium High Risk': 2, 'High Risk': 3}
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(ord_cols), 'Variable_Type'] = 'Ordinal'
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(ord_cols), 'Encoded_Values'] = str(encode)

In [ ]:
########################################################################################################################
# 8. To be encoded subsequently
########################################################################################################################
subseq_cols: list[str] = ['OutpatPrimaryDx', 'OutpatNonPrimaryDx', 'EDPrimDx']
for c in subseq_cols:
    assert c in df_enc_dict['Variable_Name'].to_list()
df_enc_dict.loc[df_enc_dict['Variable_Name'].isin(subseq_cols), 'Variable_Type'] = 'To be encoded later'

In [ ]:
########################################################################################################################
# Export the result as a new XLSX file
########################################################################################################################
df_pat_dict: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Patient')
with pd.ExcelWrriter(DICT_OUT_FILE_PATH) as writer:
    df_pat_dict.to_excel(writer, sheet_name='Patient', index=False)
    df_enc_dict.to_excel(writer, sheet_name='Encounter', index=False)